In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout,Dense,ReLU,Softmax

In [2]:
df = pd.read_csv('E:/Documents/GitHub/Express-U/data/preprocessed/canny_labels.csv',index_col=False)
df

,file_paths,labels
0,E:/Documents/GitHub/Express-U/data/preprocesse...,1
1,E:/Documents/GitHub/Express-U/data/preprocesse...,1
2,E:/Documents/GitHub/Express-U/data/preprocesse...,1
3,E:/Documents/GitHub/Express-U/data/preprocesse...,1
4,E:/Documents/GitHub/Express-U/data/preprocesse...,1
...,...,...
41995,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41996,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41997,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41998,E:/Documents/GitHub/Express-U/data/preprocesse...,Z


In [3]:
X_Train,X_Test = train_test_split(df,train_size=0.8,random_state=42,shuffle=True)

In [4]:
Train_image_generator = ImageDataGenerator(rescale=1/255,
                                        zoom_range=0.5,
                                        brightness_range=[0.6,1.0],
                                        rotation_range=20,
                                        validation_split=0.1)
Test_image_generator = ImageDataGenerator(rescale=1/255)

In [5]:
Train_Set = Train_image_generator.flow_from_dataframe(dataframe=X_Train,
                                                   x_col="file_paths",
                                                   y_col="labels",
                                                   batch_size=32,
                                                   class_mode="categorical",
                                                   color_mode="grayscale",
                                                   subset="training")

Validation_Set = Train_image_generator.flow_from_dataframe(dataframe=X_Train,
                                                   x_col="file_paths",
                                                   y_col="labels",
                                                   batch_size=32,
                                                   class_mode="categorical",
                                                   color_mode="grayscale",
                                                   subset="validation")


Test_Set = Test_image_generator.flow_from_dataframe(dataframe=X_Test,
                                                   x_col="file_paths",
                                                   y_col="labels",
                                                   batch_size=32,
                                                   class_mode="categorical",
                                                   color_mode="grayscale",
                                                   shuffle=False)

Found 30240 validated image filenames belonging to 35 classes.
Found 3360 validated image filenames belonging to 35 classes.
Found 8400 validated image filenames belonging to 35 classes.


In [6]:
Train_Set.image_shape

(256, 256, 1)

In [7]:
inputshape = (256,256,1)
outputshape = 35

model = Sequential()
model.add(Conv2D(input_shape=inputshape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(outputshape,activation="softmax"))


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 524288)            0

In [9]:
lr=0.001
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])

In [10]:
history = model.fit(Train_Set,epochs=10,steps_per_epoch=(Train_Set.image_shape[0] // 32),validation_steps=(Validation_Set.image_shape[0] // 32),batch_size=32,verbose=1,validation_data=Validation_Set)

Epoch 1/10
8/8 [==============================] - 268s 34s/step - loss: 3.5413 - accuracy: 0.0273 - val_loss: 3.4513 - val_accuracy: 0.0820
Epoch 2/10
8/8 [==============================] - 284s 37s/step - loss: 3.1595 - accuracy: 0.1602 - val_loss: 2.9538 - val_accuracy: 0.2227
Epoch 3/10
8/8 [==============================] - 260s 33s/step - loss: 2.6621 - accuracy: 0.2891 - val_loss: 2.7521 - val_accuracy: 0.2852
Epoch 4/10
8/8 [==============================] - 252s 32s/step - loss: 2.4483 - accuracy: 0.3438 - val_loss: 2.1022 - val_accuracy: 0.4414
Epoch 5/10
8/8 [==============================] - 244s 31s/step - loss: 2.1693 - accuracy: 0.4219 - val_loss: 2.0139 - val_accuracy: 0.4844
Epoch 6/10
8/8 [==============================] - 250s 32s/step - loss: 1.8732 - accuracy: 0.5195 - val_loss: 1.7951 - val_accuracy: 0.5625
Epoch 7/10
8/8 [==============================] - 247s 32s/step - loss: 1.6109 - accuracy: 0.5391 - val_loss: 1.5756 - val_accuracy: 0.5664
Epoch 8/10
8/8 [====

In [14]:
history.history

{'loss': [3.5412585735321045,
  3.1594512462615967,
  2.6620888710021973,
  2.448328971862793,
  2.169344425201416,
  1.8732231855392456,
  1.610938549041748,
  1.5306503772735596,
  0.9847385883331299,
  0.9151589274406433],
 'accuracy': [0.02734375,
  0.16015625,
  0.2890625,
  0.34375,
  0.421875,
  0.51953125,
  0.5390625,
  0.5703125,
  0.71875,
  0.734375],
 'val_loss': [3.4512529373168945,
  2.9537642002105713,
  2.7520899772644043,
  2.1021885871887207,
  2.013854503631592,
  1.795127272605896,
  1.5756263732910156,
  1.2932186126708984,
  1.0681335926055908,
  0.8646857738494873],
 'val_accuracy': [0.08203125,
  0.22265625,
  0.28515625,
  0.44140625,
  0.484375,
  0.5625,
  0.56640625,
  0.65625,
  0.69921875,
  0.7421875]}

In [15]:
model.evaluate(Test_Set,steps=(Test_Set.image_shape[0] // 32))

8/8 [==============================] - 50s 6s/step - loss: 0.2367 - accuracy: 0.9297


[0.23673881590366364, 0.9296875]

In [16]:
model.save('model_canny.h5')
model.save_weights('model_canny_weights.h5')